In [1]:
import numpy as np
import numpy.linalg as la
import scipy.linalg as sla
import matplotlib.pyplot as plt
import pandas as pd
import make_table

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-0tsliwpd because the default path (/tmp/cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
def least_sq(A, y):
    Q,R = np.linalg.qr(A)
    x = sla.solve_triangular(R,Q.T@ y)
    return x

# Baseball Team Rankings

## Part 1: A small example

In this question, you will estimate the "rankings" of baseball teams using their past game performance.
We will first do a tiny example to show how one might try to rank four teams, A,B,C,D who all played against one another in a season.

Game 1: Team A vs Team B: 1-4<br>
Game 2: Team C vs Team D: 4-3<br>
Game 3: Team A vs Team C: 9-5<br>
Game 4: Team B vs Team D: 6-8<br> 
Game 5: Team A vs Team D: 9-0<br>
Game 6: Team B vs Team C: 3-8

Now, let us suppose that the object of our ranking system is that the difference in the rankings between Team I and Team J should correspond to the point difference in a game where Team I plays Team J.<br>
This gives us, for each pair of teams, a linear equation
$$r_I - r_J = y_{IJ}$$
where $r_I$ is the ranking of Team $I$, and $y_{IJ}$ is the score difference between Team $I$ and Team $J$.

$\textbf{1.}$ Write down the matrix equation $Ar = Y$ which corresponds to this system of linear equations. Each row corresponds to one game, and each column corresponds to a team. Write the vector $Y$ such that each entry is positive. For example, for Game 1, in the since Team B won and Team A lost, we will put a "1" in the frst row in the B column, and a "-1" in the A column, and "0"'s in the rest of row 1. The score difference was 3, so a 3 will be the first entry of $y$.

$$\begin{matrix}A&B&C&D\end{matrix}$$
$$
\hspace{.6cm}\begin{matrix}Game\:1\\ Game\:2\\ Game\:3\\ Game\:4\\ Game\:5\\ Game\:6\end{matrix}\begin{pmatrix}-1&1&0&0\\ _{_{ }}&_{ }&_{ }&_{ }\\ _{ }&_{ }&_{ }&_{ }\\ &&&\\ &&&\\ &&&\end{pmatrix}\begin{pmatrix}r_A\\ r_B\\ r_C\\ r_D\end{pmatrix}=\begin{pmatrix}3\\ _{ }\\ _{ }\\ \\ \\ \end{pmatrix}
 $$
Enter the matrix <code>A</code> below as a (6,4) numpy array, and the vector <code>Y</code> as a (6, ) numpy array. Although the order of the rows doesn't matter for a system of equations, the rows of your matrix should be ordered according to the game order above for grading purposes.

The resulting matrix is called the Massey Matrix.

In [3]:
#grade
A = np.array([
    [-1,1,0,0],
    [0,0,1,-1],
    [1,0,-1,0],
    [0,-1,0,1],
    [1,0,0,-1],
    [0,-1,1,0]
])
y = np.array([3, 1, 4, 2, 9, 5]).T

In [4]:
#Check the least squares solution to your system using the function least_sq from class
r= least_sq(A,y)
print(r)

[-1.37927981e+16 -1.37927981e+16 -1.37927981e+16 -1.37927981e+16]


$\textbf{Question:}$ Are the ranking we've obtained unique? That is, is there a unique solution to the least squares system $A^TAx = A^Ty$?

$\textbf{Answer:}$ No, we can compute the rank of $A^TA$ and see that it is not full rank! Thus we don't have unique least squares rankings. That is why Python may give zero as the "least squares solution".

In [5]:
ATA = np.transpose(A) @A
print('A^TA =', ATA)
print('The rank of A^TA is', np.linalg.matrix_rank(ATA))

A^TA = [[ 3 -1 -1 -1]
 [-1  3 -1 -1]
 [-1 -1  3 -1]
 [-1 -1 -1  3]]
The rank of A^TA is 3


$\textbf{How to fix it:}$ Recall $\text{Nul(A)}=\text{Nul}(A^TA)$ and that the solution to the system $A^TAx = A^Ty$ is unique iff $\text{Nul}(A^TA)=\{0\}$. <p>

Also recall that $A^TAx=A^Ty$ has a unique solution iff $A$ is full rank. <p>
    
But this will never be true for the A we have constructed because the columns are dependent. Notice that we will always obtain a symmetric matrix (the number of games team I plays against team J is the same as the umber of games team J plays against team I). We will also always obtain linearly dependent columns because the game history of a team is completely determined by the game history of every other team.<p>
    How can we fix this?<p>
    
One possible solution is to set $\sum_j{r_j} = 0$. This adds one more constraint on the system.<p>

Adding one row of ones and a zero to $A$ does this. <p>
    
Note $v = (1,...,1)$ is always in the null space of $A$ because each row contains a $1$ and a $-1$. So $v$ is in $\text{Nul}(A^TA)$. So the sum of the columns of $A^TA$ is zero, and since $A^TA$ is symmetric, the sum of the rows must be zero also.<p>
$$(A^Ty)^Tv = y^TAv = 0,$$ so $v\in \text{Nul}(A^Ty)^T$. Thus, $\textbf{any one of the normal equations in  can be eliminated and replaced by the equation}$ $$\sum_j{r_j} = 0.$$ In other words, $\textbf{in practice, after we find $A^TA$, we can delete the last line and replace it by all ones, and change the last entry of $A^Ty$ to a zero}$

In [6]:
X = ATA
X[np.shape(X)[1]-1,:] = np.ones(np.shape(X))[np.shape(X)[1]-1,:]
p = (np.transpose(A)@y)
p[np.shape(p)[0]-1]=0
print(X)
print(p)
print("The rank of our altered matrix X is", np.linalg.matrix_rank(X))


[[ 3 -1 -1 -1]
 [-1  3 -1 -1]
 [-1 -1  3 -1]
 [ 1  1  1  1]]
[10 -4  2  0]
The rank of our altered matrix X is 4


It works! We have obtained a full rank matrix.
We can then solve this system using Least Squares and obtain unique rankings for all the teams:

In [7]:
r = la.solve(X,p)

Teams = ['Team A', 'Team B', 'Team C', 'Team D']
for i in range(len(Teams)):
    print(Teams[i],'has rank',r[i])

Team A has rank 2.5
Team B has rank -1.0000000000000002
Team C has rank 0.5
Team D has rank -2.0


## Part 2: Application to real data

Now let's try to use the same method on the real 2021 season MLB Baseball data. The following dataset was downloade from https://www.retrosheet.org/gamelogs/index.html:

In [8]:
data = pd.read_csv('GL2021.csv')
data=data.drop(labels=['Game_Date','Game_Number','Day_of_Week','Visiting_League','Visiting_Game_Number','Home_League','Home_Game_Number','Day_Night','Game_Length'],axis=1)
data.index=[''] * len(data)
print(data)

   Visiting_Team Home_Team  Visiting_Score  Home_Score
             ANA       ARI               6           5
             ANA       ARI               8           7
             ANA       ARI              10           3
             ANA       BAL              14           8
             ANA       BAL               6          10
..           ...       ...             ...         ...
             WAS       SLN               6           0
             WAS       TBA               1           3
             WAS       TBA               9           7
             WAS       TOR               5           9
             WAS       TOR               8           2

[2429 rows x 4 columns]


The columns are ordered alphabetically by the visiting team column and we've gotten rid of some extraneous columns. for ease, the data is provided to you in a numpy array <code>Table</code> with the team names replaced by a number.

In [9]:
Table = make_table.Table
Teams = make_table.Teams
print(Table)

[[0 1 6 5]
 [0 1 8 7]
 [0 1 10 3]
 ...
 [29 26 9 7]
 [29 28 5 9]
 [29 28 8 2]]


$\textbf{3. Create the Massey matrix A, and vector y}$ Now we can write a code which uses the above table to produce the massey matrix. It should contain a 1 in the column of the winning team and a -1 in the column of the losing team. We'll store this value as <code>massey_matrix</code>. 

There are 30 teams so the massey matrix should have 30 columns.

We can at the same time create the point differential vector <code>s</code>. Each entry should be the (positive) difference in the number of points between the two teams that played:
Here is an example of how we can programatically generate such a matrix.

In [11]:
massey_matrix = np.zeros(shape = (len(Table),len(Teams)))
s = np.zeros(len(Table))
for i in range(0,len(Table)):
    j = Table[i,0]
    k = Table[i,1]
    point_diff = Table[i,2] - Table[i,3]
    if point_diff >0:
        massey_matrix[i,j]=1
        massey_matrix[i,k]=-1
        s[i]=point_diff
    if point_diff<0:
        massey_matrix[i,j]=-1
        massey_matrix[i,k]=1
        s[i]=-point_diff
print(massey_matrix,s)

[[ 1. -1.  0. ...  0.  0.  0.]
 [ 1. -1.  0. ...  0.  0.  0.]
 [ 1. -1.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  1.]
 [ 0.  0.  0. ...  0.  1. -1.]
 [ 0.  0.  0. ...  0. -1.  1.]] [1. 1. 7. ... 2. 4. 6.]


$\textbf{4.}$ In order to fix the problem of rank for the least squares system, use the trick above and replace the last row of $A^TA$ with a row of ones, and the last entry of $A^Ts$ with a zero to capture the equation $\sum_{j}r_j = 0$, using <code>massey_matrix</code> as $A$.
Store the resulting matrix as <code>X</code>, and the resulting vector as <code>p</code>.

In [13]:
#grade
X = massey_matrix.T @ massey_matrix
X[np.shape(X)[1]-1,:] = np.ones(np.shape(X))[np.shape(X)[1]-1:]
p = np.transpose(massey_matrix)@s
p[np.shape(p)[0]-1]=0
# print(X)
# print(p)

Finally, solve the system $Xr = p$. Store the vector $r$ as <code>rankings</code>. You may use <code>la.solve</code> for this question.

In [15]:
#grade
rankings = la.solve(X, p)


Finally, run this cell to sort the list of rankings and print the rankings of each team!

In [16]:
df2 = pd.DataFrame(Teams, columns = ['Team'])
df2['Ranking'] = rankings
df2=df2.sort_values(by='Ranking',ascending=False)
print(df2.to_string(index=False))

Team   Ranking
 LAN  1.567014
 TBA  1.274325
 SFN  1.233793
 HOU  1.202525
 TOR  1.170958
 CHA  0.794020
 ATL  0.702687
 BOS  0.597071
 MIL  0.440930
 OAK  0.400509
 NYA  0.377939
 SDN  0.214326
 SLN -0.009017
 CIN -0.039366
 PHI -0.106722
 CLE -0.123638
 NYN -0.227051
 SEA -0.231304
 COL -0.236326
 ANA -0.338024
 DET -0.422279
 MIA -0.456290
 WAS -0.583594
 KCA -0.635421
 MIN -0.642204
 CHN -0.903748
 TEX -0.993257
 ARI -1.098826
 PIT -1.422808
 BAL -1.506221
